In [3]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import problem_unittests as tests
import tarfile


train_data_url = 'https://www.kaggle.com/c/5441/download/train.zip'
# for test: https://www.kaggle.com/c/5441/download/test.zip

train_folder_path = 'train.zip'
train_location = 'data/train.zip'

if isfile(train_location):
    zip_path = train_location
else:
    zip_path = 'train.zip'
    
class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num
        
if not isfile(zip_path):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='train dataset') as pbar:
        urlretrieve(train_data_url, zip_path, pbar.hook)
        
if not isdir(train_folder_path):
    with tarfile.open(zip_path) as tar:
        tar.extractall()
        tar.close()

tests.test_folder_path(train_folder_path)

ReadError: file could not be opened successfully

In [10]:
import tensorflow as tf
import helper

def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)
    print (image_decoded.shape)
    image_resized = tf.image.resize_images(image_decoded, [28, 28])
    return image_resized, label

train_files = helper.get_train_files()
train_labels = [s[s.rfind('/') + 1:s.index('.')] for s in train_files]

files = tf.constant(train_files)
labels = tf.constant(train_labels)

dataset = tf.data.Dataset.from_tensor_slices((files, labels))
dataset = dataset.map(_parse_function)

print(len(train_files))
print(train_files[0])
print(train_labels[0])
print(type(dataset))
# filenames = tf.constant()

(?, ?, ?)
25000
/Users/gaoyunfei/Documents/gitoc/cat_vs_dog/train/dog.8011.jpg
dog
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [13]:
batched_dataset = dataset.batch(4)
print (type(batched_dataset))
iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    print(sess.run(next_element))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
(array([[[[ 21.        ,  31.        ,  43.        ],
         [ 28.428574  ,  37.714287  ,  59.14286   ],
         [ 33.85714   ,  49.571426  ,  74.71428   ],
         ...,
         [ 40.142883  ,  59.857178  ,  86.285706  ],
         [ 53.285706  ,  76.285706  , 107.42856   ],
         [125.99982   , 161.99982   , 197.99982   ]],

        [[ 18.285713  ,  25.285713  ,  44.285713  ],
         [ 43.816326  ,  57.53061   ,  78.95918   ],
         [ 48.285713  ,  68.71428   , 101.85715   ],
         ...,
         [ 32.918346  ,  47.204052  ,  72.34693   ],
         [ 42.734734  ,  65.59188   ,  83.775536  ],
         [109.9184    , 137.55101   , 188.36734   ]],

        [[ 17.571426  ,  20.571426  ,  35.571426  ],
         [ 48.142853  ,  66.428566  ,  92.85714   ],
         [ 42.571426  ,  61.571426  ,  91.85714   ],
         ...,
         [ 59.836678  ,  76.55098   , 102.75504   ],
         [ 94.83676   , 123.551056  , 163.9